In [1]:
from itertools import chain
def process_csv(path, dirty_acceptation=25):
    series = ['ENGRPM', 'FUELUS', 'ENGTPS', 'GROILP', 'SELGEAR', 'SPEED', 'BREAKP', 'TRNAUT', 'HYDOILP']
    f = open(path, 'rt')
    csv_content = f.readlines()
    f.close()
    output = []
    if len(csv_content) > 0 and csv_content[0] == "ASCII\n":
        sep = csv_content[1].replace("\n", "")
        s = 0
        dirty = False
        ts = None
        sample = [[None for i in range(9)] for i in range(5)]
        for line in csv_content:
            line = line.replace("\n", "").split(sep)
            if not dirty:
                for i, j in enumerate(series):
                    if j in line[0]:
                        if ts is None:
                            ts = line[3]
                        if ts != line[3] and s != 4:
                            ts = line[3]
                            s += 1
                        if line[6] != "192":
                            sample[s][i] = None
#                             dirty = True
                        else:
                            sample[s][i] = float(line[5])
                        break
            if 'ENGOILP' in line[0]:
                if line[6] == "192":
                    sample = list(chain(*sample)) + [line[5]]
                else:
                    sample = list(chain(*sample)) + [None]
                if sample.count(None) < dirty_acceptation:
                    output.append(sample)
                s = 0
                dirty = False
                sample = [[None for i in range(9)] for i in range(5)]
        return output
    return output

In [2]:
import os
dataset_path = './datasets/LK3 050L'
learning_dataset = []

for year in os.listdir(dataset_path):
    for month in os.listdir('/'.join([dataset_path, year])):
        for f in os.listdir('/'.join([dataset_path, year, month])):
            if f.endswith('.csv'):
                learning_dataset = learning_dataset + process_csv('/'.join([dataset_path, year, month, f]))

In [3]:
import numpy as np
learning_dataset_np = np.array(learning_dataset, dtype=float)
means = np.nanmean(learning_dataset_np, axis=0)
means[np.isnan(means)] = 0
for i in range(means.size):
    learning_dataset_np[:,i][np.isnan(learning_dataset_np[:,i])] = means[i]

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
import time
print(time.time())
clf = LocalOutlierFactor(n_neighbors=2, novelty=True).fit(learning_dataset_np)
print(time.time())

1623539621.4641385


In [ ]:
import os
dataset_path = './datasets_incoming_accident/LK3 050L'
test_dataset = []

for year in os.listdir(dataset_path):
    for month in os.listdir('/'.join([dataset_path, year])):
        for f in os.listdir('/'.join([dataset_path, year, month])):
            if f.endswith('.csv'):
                test_dataset = test_dataset + process_csv('/'.join([dataset_path, year, month, f]))

In [ ]:
import numpy as np
test_dataset_np = np.array(test_dataset, dtype=float)
means = np.nanmean(test_dataset_np, axis=0)
means[np.isnan(means)] = 0
for i in range(means.size):
    test_dataset_np[:,i][np.isnan(test_dataset_np[:,i])] = means[i]

In [ ]:
dd = clf.predict(test_dataset_np)

In [ ]:
np.count_nonzero(dd == -1) / len(dd)

In [ ]:
dd2 = clf.predict(learning_dataset_np[:100000])

In [ ]:
np.count_nonzero(dd2 == -1) / len(dd2)

In [ ]:
import os
import numpy as np
dataset_path = './datasets_incoming_accident/LK3 050L'
test_dataset = []

for year in os.listdir(dataset_path):
    for month in os.listdir('/'.join([dataset_path, year])):
        for f in os.listdir('/'.join([dataset_path, year, month])):
            if f.endswith('.csv'):
                file_path = '/'.join([dataset_path, year, month, f])
                test_dataset = process_csv(file_path)
                if len(test_dataset) > 0:
                    test_dataset_np = np.array(test_dataset, dtype=float)
                    means = np.nanmean(test_dataset_np, axis=0)
                    means[np.isnan(means)] = 0
                    for i in range(means.size):
                        test_dataset_np[:,i][np.isnan(test_dataset_np[:,i])] = means[i]
                    dd = clf.predict(test_dataset_np)
                    anomaly_percentage = np.count_nonzero(dd == -1) * 100 // len(dd)
                    print("{}: {}% anomalii".format(file_path, anomaly_percentage))